In [1]:
# Import libraries
import torch
import pandas as pd
import numpy as np
import subprocess
import os
import shutil

from utils.helper import extract_metrics_from_output

# 1. Connecting to CUDA

Run on GPU, because running it on CPU will cost a lot of time.


I do not recommend to run it in Google Colab, because it interrupts training process.

If you are not going to use remote servers with multiple GPUs, skip this part.

In [2]:
# For CUDA making it available this works:
# pip3 install torch torchvision torchaudio

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [3]:
# Check the number of available GPUs
num_gpus = torch.cuda.device_count()
print("Number of available GPUs:", num_gpus)

Number of available GPUs: 3


In [4]:
torch.cuda.get_device_name(1)

'Tesla V100-PCIE-32GB'

In [9]:
# Index of the GPU you want to use (e.g., 0, 1, 2, etc.)
# Choose that one that is not used by other processes
gpu_index = "1"

# 2. Informer

In [ ]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/main_experiments"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [ ]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = ["5", "5", "3", "3", "3"]
seq_len = "96"
model = "Informer"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning
lr = 0.0001
n_heads = 16
e_layers = 2
d_layers = 1

# List to store the results
informer_results = []

with open(log_file_path, "w") as log_file:

  for i, country in enumerate(countries):

    log_file.write(f"\n=== Starting experiments for country: {country} ===\n")

    for pred_len in pred_lens:
      model_id = f"{country}_{seq_len}_{pred_len}"
      dataset = f"{country}_data.csv"
      
      # Arguments for the command
      command = f"""
      python {script_path} \
        --random_seed 2021 \
        --is_training 1 \
        --root_path "{data_path}" \
        --data_path "{dataset}" \
        --model_id {model_id} \
        --model "{model}" \
        --data "custom" \
        --features M \
        --seq_len {seq_len} \
        --label_len 5 \
        --pred_len {pred_len} \
        --e_layers 2 \
        --d_layers 1 \
        --factor 5 \
        --enc_in "{num_cols[i]}" \
        --dec_in "{num_cols[i]}" \
        --c_out "{num_cols[i]}" \
        --des 'Exp' \
        --train_epochs 10 \
        --patience 3 \
        --n_heads {n_heads} \
        --overlapping_windows \
        --itr 1 --batch_size 32 --learning_rate {lr}
      """
      # Log the country and prediction length
      log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

      # Run the command and capture the output
      process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

      # Capture the output in real-time
      output = []
      for line in process.stdout:
          output.append(line)
          print(line, end='')  # Print in the .ipynb cell
          log_file.write(line)  # Write to the log file

      # Wait for the process to complete
      process.wait()

      # Extract metrics from the captured output
      mse, mae = extract_metrics_from_output(output)
      
      # Delete the checkpoints folder and all its contents
      shutil.rmtree('./checkpoints' )

      # Log the extracted metrics
      log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}:\n")
      log_file.write(f"MSE: {mse}, MAE: {mae}\n")

      # Append the results to the informer_results list
      informer_results.append({
          'Country': country,
          'Pred_len': pred_len,
          'MSE': mse,
          'MAE': mae
      })

In [ ]:
# Convert the collected data into DataFrame
informer_df = pd.DataFrame(informer_results)

# Set multi-index 
informer_df.set_index(['Country', 'Pred_len'], inplace=True)
informer_df = informer_df.round(4)

# PatchTST

We separated PatchTST from Informer, because it has additional arguments. It is not so easy to modify f-string (as e. g. distionary) to unpack some arguments with if statement. Moreover, it has different parameter values.

In [ ]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = ["5", "5", "3", "3", "3"]
seq_len = "336"
model = "PatchTST"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning,but default
lr = 0.00001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2

# List to store the results
patchtst_results = []

with open(log_file_path, "w") as log_file:

  for i, country in enumerate(countries):

    log_file.write(f"\n=== Starting experiments for country: {country} ===\n")

    for pred_len in pred_lens:
      model_id = f"{country}_{seq_len}_{pred_len}"
      dataset = f"{country}_data.csv"

      # Arguments for the command
      command = f"""
      python {script_path} \
        --random_seed 2021 \
        --is_training 1 \
        --root_path "{data_path}" \
        --data_path "{dataset}" \
        --model_id {model_id} \
        --model "{model}" \
        --data "custom" \
        --features M \
        --seq_len {seq_len} \
        --label_len 5 \
        --pred_len {pred_len} \
        --e_layers 3 \
        --factor 5 \
        --enc_in "{num_cols[i]}" \
        --dec_in "{num_cols[i]}" \
        --c_out "{num_cols[i]}" \
        --des 'Exp' \
        --train_epochs 20 \
        --patience 3 \
        --n_heads {n_heads} \
        --patch_len 32 \
        --stride 16 \
        --overlapping_windows \
        --itr 1 --batch_size 64 --learning_rate {lr}
      """
      # Log the country and prediction length
      log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

      # Run the command and capture the output
      process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

      # Capture the output in real-time
      output = []
      for line in process.stdout:
          output.append(line)
          print(line, end='')  # Print in the .ipynb cell
          log_file.write(line)  # Write to the log file

      # Wait for the process to complete
      process.wait()

      # Extract metrics from the captured output
      mse, mae = extract_metrics_from_output(output)

      # Log the extracted metrics
      log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}:\n")
      log_file.write(f"MSE: {mse}, MAE: {mae}\n")

      # Append the results to the list
      patchtst_results.append({
          'Country': country,
          'Pred_len': pred_len,
          'MSE': mse,
          'MAE': mae
      })

In [ ]:
# Convert the collected data into DataFrame
patchtst_df = pd.DataFrame(patchtst_results)

# Set multi-index 
patchtst_df.set_index(['Country', 'Pred_len'], inplace=True)
patchtst_df = patchtst_df.round(4)